In [1]:
# Importaciones  
import numpy as np  
import tensorflow as tf  
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Flatten, Conv2D, Dense, MaxPooling2D, Lambda  
from tensorflow.keras.preprocessing.image import ImageDataGenerator  
from pathlib import Path  
import sys  
import pandas as pd

In [2]:
# Añadir el directorio raíz del proyecto a sys.path
module_path = Path('/Users/Cesar/Desktop/Proyecto-CSG/').resolve()
if module_path not in sys.path:
    sys.path.append(str(module_path))

# Añadir el directorio que contiene 'new_spectral_metric' a sys.path
new_spectral_metric_path = module_path / '/Users/Cesar/Desktop/Proyecto-CSG/new_spectral_metric/'
if new_spectral_metric_path not in sys.path:
    sys.path.append(str(new_spectral_metric_path))

from new_spectral_metric.new_estimator import CumulativeGradientEstimator  
from new_spectral_metric.new_visualize import make_graph  



In [4]:
# Lectura de dataset

# Configuración del directorio del dataset  
dataset_dir = Path('../Datasets/Cats and dogs - train')  
cats_dir = dataset_dir / 'cats'  
dogs_dir = dataset_dir / 'dogs'  

# Contar imágenes  
total_images = sum(1 for _ in dataset_dir.glob('*/*.jpg'))  
total_cats = sum(1 for _ in cats_dir.glob('*.jpg'))  
total_dogs = sum(1 for _ in dogs_dir.glob('*.jpg'))  

print(f"Total imágenes: {total_images}")  
print(f"Total imágenes de gatos: {total_cats}")  
print(f"Total imágenes de perros: {total_dogs}")  

Total imágenes: 696
Total imágenes de gatos: 348
Total imágenes de perros: 348


In [5]:
# Dimensiones de la imagen y tamaño del batch  
img_height = 32  
img_width = 32 
batch_size = total_images  # Configurar un batch size razonable  

In [6]:

datagen = ImageDataGenerator(rescale=1.0/255)  

data_flow = datagen.flow_from_directory(  
    dataset_dir,  
    target_size=(img_height, img_width),  
    batch_size=batch_size,  
    class_mode='binary',  # Forma binaria ya que solamente tienes dos clases  
    shuffle=False  # No mezclar si no es necesario para la tarea actual  
)  

Found 696 images belonging to 2 classes.


In [7]:
# Obtener los nombres de las clases de los datos  
class_names = data_flow.class_indices.keys()  
num_classes = len(class_names)  
print(f"Número de clases: {num_classes}")  
print(f"Los nombres de las clases son: {class_names}")  


Número de clases: 2
Los nombres de las clases son: dict_keys(['cats', 'dogs'])


In [8]:
# Convertir las imágenes a arrays numpy para visualización  
images, labels = next(data_flow)  
assert len(images) == batch_size  # Verifica el tamaño del batch  
images /= 255.0  # Escala las características al rango [0, 1]  
print(images.shape)  


(696, 32, 32, 3)


In [9]:
# Preprocesamiento de datos para CumulativeGradientEstimator  
fully_dataset = images.reshape((images.shape[0], -1))  
fully_labels = labels.reshape(labels.shape[0],) 

In [11]:
# Inicializar y ajustar el estimador  
estimator = CumulativeGradientEstimator(M_sample=348, k_nearest=10)  
estimator.fit(data=fully_dataset, target=fully_labels)  
csg = estimator.csg  
print(f"CSG calculado: {csg}")  

make_graph(estimator.difference, title=f"Comparativa entre clases, CSG: {csg}", classes=list(class_names))  

TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [17]:
# Ordena los pares de clases en orden de similitud ascendente (menos similar primero)
pairs = list(zip(*np.unravel_index(np.argsort(estimator.W, axis=None), estimator.W.shape)))
pairs = [(original,corrupto) for original,corrupto in pairs if original != corrupto]

similitud_de_clases = pairs[0]
original, corrupto = similitud_de_clases  # Pares de clases menos similares

print("Calculo de similitud entre clases")
lst = []
for idx, (original,corrupto) in enumerate(pairs[::2][:1]):
    lst.append({"Clases a comparar" : f"{class_names[original]} <> {class_names[corrupto]}", "Similitud": estimator.W[original,corrupto]})
print(pd.DataFrame(lst))
print(estimator.W.shape)

Calculo de similitud entre clases
  Clases a comparar  Similitud
0      cats <> dogs   0.942377
(2, 2)


In [19]:
for class_name, indices in estimator.class_indices.items():
    print(f"Clase: {class_names}, Número de muestras: {len(indices)}")


Clase: ['cats', 'dogs'], Número de muestras: 348
Clase: ['cats', 'dogs'], Número de muestras: 348


In [24]:

# Obtener todos los nombres de archivo en el dataset
filenames = [str(f) for f in dataset_dir.glob('*/*.jpg')]

# Crear un mapeo de índices a nombres de archivo (solo el nombre de archivo, sin la ruta)
filenames_dict = {i: f.name for i, f in enumerate(dataset_dir.glob('*/*.jpg'))}

def show_least_similar_using_P(source_intent, target_intent, estimator, class_names, filenames_dict):
    print(f"{class_names[source_intent]} <> {class_names[target_intent]}")
    
    # Obtener la matriz de similitud de las muestras entre las clases
    P = estimator.P[(source_intent, target_intent)]
    
    # Ordenar los índices según las similitudes (de menor a mayor)
    least_similar_indices = np.argsort(P, axis=None)
    
    # Conjunto para llevar un registro de los archivos ya mostrados
    shown_files = set()
    count = 0
    
    for idx in least_similar_indices:
        if count >= 10:
            break
        
        # Convertir el índice plano a índice 2D
        idx_2d = np.unravel_index(idx, P.shape)
        
        # Obtener los índices de las muestras en el dataset
        source_idx = estimator.class_indices[source_intent][idx_2d[0]]
        target_idx = estimator.class_indices[target_intent][idx_2d[1]]
        
        # Verificar que los índices estén dentro de los límites del dataset
        if source_idx < len(filenames_dict) and target_idx < len(filenames_dict):
            source_filename = filenames_dict[int(source_idx)]
            target_filename = filenames_dict[int(target_idx)]
            
            if source_filename not in shown_files and target_filename not in shown_files:
                shown_files.add(source_filename)
                shown_files.add(target_filename)
                
                value = P[idx_2d]
                print(f"\tNombre del archivo: {source_filename}, Clase: {class_names[source_intent]}, Similitud: {value:.4f}")
                count += 1

# Suponiendo que `pairs` contiene los índices de las clases en el formato [(0, 1)]
pairs = [(0, 1)]

first_pair = pairs[0]

# Mostrar los menos similares usando la matriz de pares (P)
show_least_similar_using_P(first_pair[0], first_pair[1], estimator, class_names, filenames_dict)
show_least_similar_using_P(first_pair[1], first_pair[0], estimator, class_names, filenames_dict)
for class_name, indices in estimator.class_indices.items():
    print(f"Clase: {class_name}, Número de muestras: {len(indices)}")


cats <> dogs
	Nombre del archivo: cat_224.jpg, Clase: cats, Similitud: 0.0674
	Nombre del archivo: cat_96.jpg, Clase: cats, Similitud: 0.1821
	Nombre del archivo: cat_435.jpg, Clase: cats, Similitud: 0.1844
	Nombre del archivo: cat_369.jpg, Clase: cats, Similitud: 0.1865
	Nombre del archivo: cat_50.jpg, Clase: cats, Similitud: 0.1887
	Nombre del archivo: cat_109.jpg, Clase: cats, Similitud: 0.2088
	Nombre del archivo: cat_546.jpg, Clase: cats, Similitud: 0.2406
	Nombre del archivo: cat_101.jpg, Clase: cats, Similitud: 0.2505
	Nombre del archivo: cat_303.jpg, Clase: cats, Similitud: 0.2658
	Nombre del archivo: cat_420.jpg, Clase: cats, Similitud: 0.3095
dogs <> cats
	Nombre del archivo: dog_403.jpg, Clase: dogs, Similitud: 0.0674
	Nombre del archivo: dog_505.jpg, Clase: dogs, Similitud: 0.1821
	Nombre del archivo: dog_197.jpg, Clase: dogs, Similitud: 0.1844
	Nombre del archivo: dog_412.jpg, Clase: dogs, Similitud: 0.1865
	Nombre del archivo: dog_344.jpg, Clase: dogs, Similitud: 0.1887
	

In [37]:
# Acceder a la matriz M después de ajustar el modelo
M = estimator.M

# Ordenar los índices según las similitudes (de menor a mayor)
least_similar_indices = np.argsort(M, axis=None)

# Conjunto para llevar un registro de los archivos ya mostrados
shown_files = set()
count = 0

# Crear un diccionario de nombres de archivo y clases
filenames_dict = {i: f.name for i, f in enumerate(dataset_dir.glob('*/*.jpg'))}
class_names_list = ['cats' if label == 0 else 'dogs' for label in fully_labels]

# Mostrar las 10 muestras menos similares
for idx in least_similar_indices:
    if count >= 10:
        break

    # Convertir el índice plano a índice 2D
    idx_2d = np.unravel_index(idx, M.shape)

    # Obtener los índices de las muestras en el dataset
    source_idx = idx_2d[0]
    target_idx = idx_2d[1]

    # Verificar que los índices estén dentro de los límites del dataset
    if source_idx < len(filenames_dict) and target_idx < len(filenames_dict):
        source_filename = filenames_dict[int(source_idx)]
        target_filename = filenames_dict[int(target_idx)]

        if (source_filename, target_filename) not in shown_files and (target_filename, source_filename) not in shown_files:
            shown_files.add((source_filename, target_filename))
            shown_files.add((target_filename, source_filename))

            value = M[idx_2d]
            print(f"\tNombre del archivo: {source_filename}, Clase: {class_names_list[source_idx]}, Similitud: {value:.4f}")
            print(f"\tNombre del archivo: {target_filename}, Clase: {class_names_list[target_idx]}, Similitud: {value:.4f}")
            count += 1

	Nombre del archivo: cat_544.jpg, Clase: cats, Similitud: 0.0508
	Nombre del archivo: cat_224.jpg, Clase: cats, Similitud: 0.0508
	Nombre del archivo: dog_403.jpg, Clase: dogs, Similitud: 0.0674
	Nombre del archivo: cat_224.jpg, Clase: cats, Similitud: 0.0674
	Nombre del archivo: dog_324.jpg, Clase: dogs, Similitud: 0.0675
	Nombre del archivo: cat_224.jpg, Clase: cats, Similitud: 0.0675
	Nombre del archivo: cat_224.jpg, Clase: cats, Similitud: 0.0693
	Nombre del archivo: cat_387.jpg, Clase: cats, Similitud: 0.0693
	Nombre del archivo: cat_224.jpg, Clase: cats, Similitud: 0.0693
	Nombre del archivo: cat_446.jpg, Clase: cats, Similitud: 0.0693
	Nombre del archivo: cat_224.jpg, Clase: cats, Similitud: 0.0694
	Nombre del archivo: dog_546.jpg, Clase: dogs, Similitud: 0.0694
	Nombre del archivo: cat_224.jpg, Clase: cats, Similitud: 0.0700
	Nombre del archivo: cat_203.jpg, Clase: cats, Similitud: 0.0700
	Nombre del archivo: cat_224.jpg, Clase: cats, Similitud: 0.0702
	Nombre del archivo: cat_